In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('deaths_and_infections.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
new_deaths=np.array(df['new_deaths'])
death_cumul=np.array([sum(new_deaths[:i]) for i in range(len(new_deaths))])
dates_of_pandemic=np.arange(len(new_deaths))

In [ ]:
def IS(interval : tuple, point : float, alpha: float) -> float: 
    assert interval[0] < interval[1]
    assert alpha >= 0
    assert alpha <= 1
    l=interval[0]
    u = interval[1]
    dispersion = u-l
    if point < l : 
        underprediction= (2/alpha)*(l-point)
    else: 
        underprediction=0
    if point > u :
        overprediction = (2/alpha)*(point-u)
    return underprediction + overprediction + dispersion


In [ ]:
def WIS(prediction: float, interval : tuple, point_of_evaluation : float, alphas: list, weights: list) -> float:
    """

    WIS computes the Weighted Interval Score of a model that predicts a point and a list of confidence intervals.
    The fuction taks as an input a prediction, a list of confidence intervals of precision alpha, a list of weights to apply to the different intervals and a point to evaluate the prediction on.
    
    """ 
    assert interval[0] < interval[1]
    assert all([alpha >= 0 for alpha in alphas])
    assert all([alpha <= 1 for alpha in alphas])
    K = len(alphas)
    loss=0
    for k in range(1, K): 
        loss += weights[k]*IS(interval, point_of_evaluation, alphas[k])
    loss += weights[0]* abs(prediction - point_of_evaluation)
    return loss


In [ ]:
def evaluate_model(model: Model, data: np.array, alphas: list, evaluation_point_indexs: list, reach: int, weights: list) -> float: 
    
    loss=0

    for index in evaluation_point_indexs: 
        model.train(data[:index])
        prediction, intervals = model.predict(reach, alphas) 

        for i in range(len(intervals)):
            interval = intervals[i] 
            loss+=WIS(prediction=prediction, interval = interval, point_of_evaluation = data[index+range], alphas = alphas , weights = weights)
    
    return loss / len(evaluation_point_indexs) # average loss over all evaluation points
